In [1]:
# install requirements
!pip install diffusers einops gradio typing_extensions  fastapi==0.100.0

# install ip-adapter
%cd /content
!git clone https://github.com/sagiodev/IP-Adapter-Negative.git

# download the models
%cd IP-Adapter-Negative
%mkdir models
%cd models
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin
%mkdir image_encoder
%cd image_encoder
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/pytorch_model.bin
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/config.json
%cd /content/IP-Adapter-Negative

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 880.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.9 MB/s 

In [1]:
# Run Negative IP adapter demo
share_public = True # whether creates a public link

import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipelineLegacy, DDIMScheduler, AutoencoderKL
from PIL import Image

from ip_adapter import IPAdapterPlus, IPAdapter
base_model_path = "SG161222/Realistic_Vision_V4.0_noVAE"
vae_model_path = "stabilityai/sd-vae-ft-mse"
image_encoder_path = "models/image_encoder"
ip_ckpt = "models/ip-adapter-plus_sd15.bin"
device = "cuda"
noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)




import random
ip_adapters_list = ['IP Adapter', 'IP Adapter Plus']
def generate(ip_adapter, image,
             neg_image,
             scale,
             scale_start,
             scale_stop,
             neg_scale,
             scale_neg_start,
             scale_neg_stop,
             prompt,
             negative_prompt,
             num_samples,
             steps,
             seed):
    if image is None and neg_image is None:
      scale =0
      neg_scale = 0
      image = Image.open("assets/images/statue.png")

    if seed == -1:
      seed = random.randint(0, 1e12)

    if ip_adapter == 'IP Adapter':
      # load ip-adapter
      ip_ckpt = "models/ip-adapter_sd15.bin"
      ip_model = IPAdapter(pipe, image_encoder_path, ip_ckpt, device)
    elif ip_adapter == 'IP Adapter Plus':
      # load ip-adapter
      ip_ckpt = "models/ip-adapter-plus_sd15.bin"
      ip_model = IPAdapterPlus(pipe, image_encoder_path, ip_ckpt, device, num_tokens=16)
    else:
      raise ValueError('IP adapter %s is undefined.'%ip_adapter)

    images = ip_model.generate(pil_image=image,
                           negative_pil_image=neg_image,
                           prompt = prompt,
                           negative_prompt= negative_prompt,
                           num_samples=num_samples,
                           scale=scale,  # weight for image prompt
                           scale_start= scale_start,
                           scale_stop= scale_stop,
                           scale_neg = neg_scale, # weight for negative image prompt
                           scale_neg_start = scale_neg_start,
                           scale_neg_stop = scale_neg_stop,
                           num_inference_steps=steps,
                           seed=seed)
    print(images)
    return images


shortcut_js = """
<script>
function shortcuts(e) {

    if ((event.keyCode == 10 || event.keyCode == 13) && event.ctrlKey) {
        document.getElementById("generate-button").click();
    }
}
document.addEventListener('keyup', shortcuts, false);
</script>
"""

import gradio as gr
with gr.Blocks(head=shortcut_js) as demo:
  resized_image_file = gr.File(visible=False)

  gr.Markdown("# Stable Diffuson Negative Image Prompt")
  with gr.Row():
    with gr.Column():
      with gr.Row():
        with gr.Column():
          prompt = gr.Textbox(label="Prompt", value = "a house")
          scale = gr.Slider(0, 2, value=1, label="Image prompt weight")
          scale_start = gr.Slider(0, 1, value=0, label="Starting step")
          scale_stop = gr.Slider(0, 1, value=1, label="Ending step")
          imagePrompt = gr.Image(label="Image Prompt", type = "pil")
        with gr.Column():
          negativePrompt = gr.Textbox(label="Negative Prompt", value = "nsfw")
          negativeScale = gr.Slider(0, 2, value=1, label="Negative Image prompt weight")
          negativeScale_start = gr.Slider(0, 1, value=0, label="Starting step")
          negativeScale_stop = gr.Slider(0, 1, value=1, label="Ending step")
          negativeImagePrompt = gr.Image(label="Negative Image Prompt", type = "pil")
      with gr.Accordion(label="Advanced options", open=False):
        ip_adapter = gr.Dropdown(choices = ip_adapters_list, value = ip_adapters_list[0], label = "IP Adapter", interactive = True)
        numImages = gr.Number(precision=0, label="Number of images", value=1)
        steps = gr.Number(precision=0, label="Number of steps", value=25)
        seed = gr.Number(precision=0, label="Seed", value=-1)
    with gr.Column():
      btn = gr.Button("Generate", elem_id="generate-button")
      imagesOut = gr.Gallery(object_fit="contain", height="auto", selected_index = 0, preview = True)


  inputs = [ip_adapter, imagePrompt, negativeImagePrompt, scale, scale_start, scale_stop, negativeScale, negativeScale_start, negativeScale_stop, prompt, negativePrompt, numImages, steps, seed]
  outputs = imagesOut
  btn.click(generate, inputs=inputs, outputs=outputs)
  demo.queue().launch(debug=True, share=share_public, inline=False, show_error=True)


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
negative image


  0%|          | 0/25 [00:00<?, ?it/s]

[<PIL.Image.Image image mode=RGB size=512x512 at 0x268F55BA6E0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x268F55BA530>, <PIL.Image.Image image mode=RGB size=512x512 at 0x268F55B9030>, <PIL.Image.Image image mode=RGB size=512x512 at 0x268F55BA620>]
negative image


  0%|          | 0/25 [00:00<?, ?it/s]

...

[<PIL.Image.Image image mode=RGB size=512x512 at 0x26AEA893250>, <PIL.Image.Image image mode=RGB size=512x512 at 0x26AEA892DD0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x26AEA892FE0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x26AEA892980>]
positive and negative image


  0%|          | 0/25 [00:00<?, ?it/s]

[<PIL.Image.Image image mode=RGB size=512x512 at 0x268F55A57E0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x268F55A5A50>, <PIL.Image.Image image mode=RGB size=512x512 at 0x268F55AD630>, <PIL.Image.Image image mode=RGB size=512x512 at 0x268F55ADEA0>]
positive and negative image


  0%|          | 0/25 [00:00<?, ?it/s]

[<PIL.Image.Image image mode=RGB size=512x512 at 0x268F52C28C0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x268F52C1D20>, <PIL.Image.Image image mode=RGB size=512x512 at 0x268F52C1ED0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x268F52C32E0>]
Keyboard interruption in main thread... closing server.
